## インストール

In [ ]:
!pip install -q --no-warn-conflicts "colab-ai-bridge[langchain] @ git+https://github.com/drillan/colab-ai-bridge"

## 使い方

### 基本的な使い方

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# モデルの作成（セットアップは自動で完了します）
model = ColabLangChainModel()

# モデルの実行
response = model.invoke("フランスの首都は？")
print(response.content)

### モデルの選択

利用可能なモデルを確認：

In [ ]:
from google.colab import ai

ai.list_models()

特定のモデルを使用：

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# Gemini 2.5 Flash Liteを使用
model = ColabLangChainModel("google/gemini-2.5-flash-lite")

### 構造化出力

LangChain の Messages API を使用して構造化されたプロンプトを作成：

In [ ]:
from pydantic import BaseModel

class City(BaseModel):
    name: str
    country: str
    population: int

model = ExtendedColabLangChainModel()
structured_model = model.with_structured_output(City)

result = structured_model.invoke("東京について教えて")
print(f"{result.name}, {result.country}, 人口: {result.population:,}")

## サンプル

### サンプル 1: LCEL - 複数チェーンの組み合わせ

LangChain Expression Language (LCEL) を使った宣言的なチェーン構築

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ColabLangChainModel()

# 複数のチェーンを組み合わせる
summary_prompt = ChatPromptTemplate.from_template(
    "以下のテキストを3文で要約してください：\n\n{text}"
)

translate_prompt = ChatPromptTemplate.from_template(
    "以下の日本語を英語に翻訳してください：\n\n{text}"
)

# チェーン1: 要約
summary_chain = summary_prompt | model | StrOutputParser()

# チェーン2: 要約 → 翻訳
translate_chain = (
    {"text": summary_chain}
    | translate_prompt
    | model
    | StrOutputParser()
)

# 実行
text = """
LangChainは、言語モデルを使用したアプリケーションを構築するためのフレームワークです。
プロンプトテンプレート、チェーン、エージェント、メモリなどの機能を提供し、
複雑なAIアプリケーションを簡単に構築できます。
LCELという宣言的な構文により、複数の処理を簡潔につなげることができます。
"""

result = translate_chain.invoke({"text": text})
print("要約 → 翻訳結果:")
print(result)

### サンプル 2: 会話履歴管理

MessagesPlaceholder を使った文脈を考慮した会話

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

model = ColabLangChainModel()

# 会話履歴を含むプロンプトテンプレート
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは親切なアシスタントです。会話の文脈を理解して応答してください。"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | model

# 会話履歴
history = [
    HumanMessage(content="私の名前は太郎です"),
    AIMessage(content="はじめまして、太郎さん。よろしくお願いします。"),
    HumanMessage(content="今日は良い天気ですね"),
    AIMessage(content="そうですね。良い天気で気持ちが良いですね。"),
]

# 履歴を踏まえた質問
result = chain.invoke({"history": history, "input": "私の名前を覚えていますか？"})
print(result.content)

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
import json


class EmailAddress(BaseModel):
    email: str = Field(pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
    domain: str


model = ColabLangChainModel()

messages = [
    SystemMessage(content="あなたは構造化されたデータを JSON 形式で返すアシスタントです。"),
    HumanMessage(
        content=f"""example@gmail.com のメールアドレス情報を以下のスキーマで抽出してください：
{EmailAddress.model_json_schema()}

JSON のみを返してください。"""
    ),
]

response = model.invoke(messages)
email_data = json.loads(response.content)
email = EmailAddress(**email_data)
print(f"メール: {email.email}")
print(f"ドメイン: {email.domain}")

from pydantic import BaseModel

class Book(BaseModel):
    title: str
    author_name: str
    year: int
    genres: list[str]

model = ExtendedColabLangChainModel()
structured_model = model.with_structured_output(Book)

result = structured_model.invoke("村上春樹の『ノルウェイの森』について教えて")
print(f"タイトル: {result.title}")
print(f"著者: {result.author_name}")
print(f"出版年: {result.year}")
print(f"ジャンル: {', '.join(result.genres)}")

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel
import json


class Book(BaseModel):
    title: str
    author_name: str
    year: int
    genres: list[str]


model = ColabLangChainModel()

messages = [
    SystemMessage(content="あなたは文学の専門家です。本の情報を JSON 形式で提供してください。"),
    HumanMessage(
        content=f"""村上春樹の『ノルウェイの森』について、以下のスキーマで情報を返してください：
{Book.model_json_schema()}

JSON のみを返してください。"""
    ),
]

response = model.invoke(messages)
book_data = json.loads(response.content)
book = Book(**book_data)
print(f"タイトル: {book.title}")
print(f"著者: {book.author_name}")
print(f"出版年: {book.year}")
print(f"ジャンル: {', '.join(book.genres)}")

### サンプル 3: プロンプトテンプレートの使用

LangChain のプロンプトテンプレートで動的な値を渡す

### サンプル 4: バッチ処理

複数の入力を一度に処理して効率化

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.prompts import ChatPromptTemplate

model = ColabLangChainModel()

prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは翻訳アシスタントです。"),
    ("human", "{text}を英語に翻訳してください")
])

chain = prompt | model

# 複数の入力を一度に処理
inputs = [
    {"text": "おはようございます"},
    {"text": "ありがとうございます"},
    {"text": "さようなら"},
]

results = chain.batch(inputs)
for i, result in enumerate(results, 1):
    print(f"{i}. {result.content}")

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from colab_ai_bridge.core.config import ModelConfig
from langchain_core.prompts import ChatPromptTemplate
from dataclasses import dataclass


@dataclass
class UserProfile:
    name: str
    age: int
    interests: list[str]


model = ColabLangChainModel()

user = UserProfile(name="太郎", age=25, interests=["プログラミング", "機械学習", "読書"])

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーのプロフィールに基づいて、パーソナライズされた提案をしてください。"),
        (
            "human",
            """ユーザー情報:
- 名前: {name}
- 年齢: {age}
- 興味: {interests}

おすすめの学習リソースを3つ教えてください""",
        ),
    ]
)

chain = prompt | model
result = chain.invoke(
    {"name": user.name, "age": user.age, "interests": ", ".join(user.interests)}
)
print(result.content)

### サンプル 5: 温度パラメータによる出力の違い

異なる温度設定での出力比較

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# 低温度（正確な出力）
config_precise = ModelConfig(temperature=0.3)
model_precise = ColabLangChainModel(config=config_precise)

# 高温度（創造的な出力）
config_creative = ModelConfig(temperature=1.0)
model_creative = ColabLangChainModel(config=config_creative)

prompt = "AIについて一文で説明してください"

print("【正確な出力 (temperature=0.3)】")
result1 = model_precise.invoke(prompt)
print(result1.content)
print()

print("【創造的な出力 (temperature=1.0)】")
result2 = model_creative.invoke(prompt)
print(result2.content)

### サンプル 6: ストリーミング出力

リアルタイムでレスポンスをストリーミング

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

model = ColabLangChainModel()

print("ストリーミング出力:")
for chunk in model.stream("日本の四季について説明してください"):
    print(chunk.content, end="", flush=True)
print()